In [1]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/var/folders/c4/kvf3th_j15lf5cq2pybvfgjm0000gn/T/ipykernel_69898/3590037574.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/Users/rhy/Library/CloudStorage/OneDrive-ArizonaStateUniversity/SelfStudy/Agentic & Gen AI with Cloud/agenticai/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dotenv import load_dotenv
load_dotenv()
import os
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')

In [3]:
len(embeddings.embed_query("Hello world"))

384

In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

In [5]:
len(embeddings.embed_query("Hello world"))

768

In [6]:
from pinecone import Pinecone

os.environ['PINECONE_API_KEY'] = os.getenv("PINECONE_API_KEY")

In [7]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

In [9]:
from pinecone import ServerlessSpec

#### Servlerless means that the server will be managed by the cloud Provider

In [14]:
index_name = "rhy-genai"

In [15]:
pc.has_index(index_name)

False

### Creating an Index

In [16]:
if not pc.has_index(index_name):
    pc.create_index(name=index_name, dimension=768, metric="cosine", spec=ServerlessSpec(cloud="aws",region="us-east-1"))

### Loading the index

In [17]:
index = pc.Index(index_name)

In [18]:
from langchain_pinecone import PineconeVectorStore

In [19]:
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
vector_store.similarity_search("What is Langchain?", k=3)

[]